In [ ]:
import math
import os
import glob

import ROOT
from hepdata_lib import Submission, Table, Variable, Uncertainty, RootFileReader


# ----------------------------------------------------------------------
# Basic config (edit these)
# ----------------------------------------------------------------------

ROOT_FILENAME = "input/ZNuNuGPrePostFitPostFitEBEE.root"
OUTPUT_DIR = "hepdata_output"

# Optional: text file containing the full paper abstract (plain text or LaTeX)
# If you create such a file, set the path here; otherwise leave as None.
ABSTRACT_FILE = None  # e.g. "abstract_zgamma_smp22009.txt"

OBSERVABLE_NAME = r"$p_{T}^{\gamma}$"
OBSERVABLE_UNITS = "GeV"

USE_GARWOOD = True
POISSON_CL = 0.6827  # ~1 sigma

CM_ENERGY_GEV = 13000.0  # 13 TeV
PAPER_TITLE = (
    "Measurement of the $Z\\gamma$ production cross section and search for "
    "anomalous neutral triple gauge couplings in pp collisions at $\\sqrt{s}=13$ TeV"
)
CMS_ANALYSIS_ID = "CMS SMP-22-009"

# Region configuration: mapping from ROOT directories/histograms to table meta
REGIONS = [
    {
        "name": "Figure4_EB_pTgamma",
        "location": "Figure 4 (left)",
        "description": (
            "Post-fit reconstruction-level photon transverse momentum $p_{T}^{\\gamma}$ "
            "distribution in the ECAL barrel (EB) signal region for the full Run-2 CMS "
            "dataset (138 fb$^{-1}$ at $\\sqrt{s}=13$ TeV). Black points labelled "
            "'Observed' show the data with Poisson statistical uncertainties. The "
            "background expectation is decomposed into the components listed as "
            "dependent variables in this table. In the published figure, the "
            "components labelled here as fiducial $Z\\gamma$, $W+\\gamma$, ECAL "
            "spikes and misidentified electrons $e\\to\\gamma$ appear explicitly "
            "in the legend. The components labelled here as jet fakes, "
            "out-of-acceptance $Z(\\nu\\nu)\\gamma$ and minor backgrounds are "
            "drawn in the stack but grouped together and shown as 'Other' in the "
            "legend. The variable 'Total background' gives the sum of all non-signal "
            "background components evaluated at their post-fit yields. The variable "
            "'Predicted' gives the full post-fit model prediction (signal plus all "
            "background components), with uncertainties propagated from the fit "
            "covariance."
        ),
        "root_dir": "postfit/EB_SR",
        "data_hist": "data_obs",
        "primary_mc": [
            (r"Fiducial $Z\gamma$", "mergedFiducialZNuNuG"),
            (r"$W+\gamma$",         "mergedWLNuG"),
            ("Spikes",              "ECAL_spikes"),
            (r"$e\to\gamma$",       "eleFakes"),
        ],
        "other_mc": [
            ("Jet fakes",                         "jetFakes"),
            (r"Out-of-acceptance $Z(\nu\nu)\gamma$", "ooaFixed"),
            ("Minor backgrounds",                 "minor_bkg"),
        ],
        "extra_mc": [
            ("Total background", "TotalBkg"),
            ("Predicted",        "TotalProcs"),
        ],
    },
    {
        "name": "Figure4_EE_pTgamma",
        "location": "Figure 4 (right)",
        "description": (
            "Post-fit reconstruction-level photon transverse momentum $p_{T}^{\\gamma}$ "
            "distribution in the ECAL endcap (EE) signal region for the full Run-2 CMS "
            "dataset (138 fb$^{-1}$ at $\\sqrt{s}=13$ TeV). Black points labelled "
            "'Observed' show the data with Poisson statistical uncertainties. The "
            "background expectation is decomposed into the components listed as "
            "dependent variables in this table. In the published figure, the "
            "components labelled here as fiducial $Z\\gamma$, $W+\\gamma$, beam "
            "halo and misidentified electrons $e\\to\\gamma$ appear explicitly "
            "in the legend. The components labelled here as jet fakes, "
            "out-of-acceptance $Z(\\nu\\nu)\\gamma$ and minor backgrounds are "
            "drawn in the stack but grouped together and shown as 'Other' in the "
            "legend. The variable 'Total background' gives the sum of all non-signal "
            "background components evaluated at their post-fit yields. The variable "
            "'Predicted' gives the full post-fit model prediction (signal plus all "
            "background components), with uncertainties propagated from the fit "
            "covariance."
        ),
        "root_dir": "postfit/EE_SR",
        "data_hist": "data_obs",
        "primary_mc": [
            (r"Fiducial $Z\gamma$", "mergedFiducialZNuNuG"),
            (r"$W+\gamma$",         "mergedWLNuG"),
            (r"$e\to\gamma$",       "eleFakes"),
            ("Beam halo",           "beamHalo"),
        ],
        "other_mc": [
            ("Jet fakes",                         "jetFakes"),
            (r"Out-of-acceptance $Z(\nu\nu)\gamma$", "OOAfixed"),
            ("Minor backgrounds",                 "minor_bkg"),
        ],
        "extra_mc": [
            ("Total background", "TotalBkg"),
            ("Predicted",        "TotalProcs"),
        ],
    },
]


# ----------------------------------------------------------------------
# Helpers
# ----------------------------------------------------------------------

def poisson_garwood_intervals(values, cl=POISSON_CL):
    alpha = 1.0 - cl
    err_down, err_up = [], []
    for y in values:
        n = float(y)
        if n < 0:
            raise ValueError(f"Negative data bin content: {n}")
        if n == 0.0:
            low = 0.0
        else:
            low = 0.5 * ROOT.Math.chisquared_quantile(alpha / 2.0, 2.0 * n)
        up = 0.5 * ROOT.Math.chisquared_quantile_c(alpha / 2.0, 2.0 * (n + 1.0))
        err_down.append(n - low)
        err_up.append(up - n)
    return err_down, err_up


def make_mc_variable(label, hist_dict):
    v = Variable(label, is_independent=False, is_binned=False)
    v.values = hist_dict["y"]
    dy = hist_dict.get("dy")
    if dy is not None:
        if len(dy) > 0 and isinstance(dy[0], tuple):
            unc = Uncertainty("stat+syst", is_symmetric=False)
            unc.values = dy
        else:
            unc = Uncertainty("stat+syst", is_symmetric=True)
            unc.values = dy
        v.add_uncertainty(unc)
    return v


def make_data_variable(label, hist_dict):
    y = hist_dict["y"]
    v = Variable(label, is_independent=False, is_binned=False)
    v.values = y

    if USE_GARWOOD:
        err_down, err_up = poisson_garwood_intervals(y)
        asym = [(-dn, +up) for dn, up in zip(err_down, err_up)]
        unc = Uncertainty("stat", is_symmetric=False)
        unc.values = asym
    else:
        errs = [math.sqrt(val) if val >= 0.0 else 0.0 for val in y]
        unc = Uncertainty("stat", is_symmetric=True)
        unc.values = errs

    v.add_uncertainty(unc)
    return v


def read_hist_1d(reader, dir_path, hist_name):
    """Robust path handling for ROOT histograms."""
    tried = []
    path1 = f"{dir_path}/{hist_name}" if dir_path else hist_name
    tried.append(path1)
    try:
        return reader.read_hist_1d(path1)
    except Exception:
        if dir_path and "/" in dir_path:
            tail = dir_path.split("/", 1)[-1]
            path2 = f"{tail}/{hist_name}"
            tried.append(path2)
            try:
                return reader.read_hist_1d(path2)
            except Exception as e2:
                raise RuntimeError(
                    f"Could not find histogram '{hist_name}' in ROOT file. "
                    f"Tried paths: {tried}"
                ) from e2
        else:
            raise RuntimeError(
                f"Could not find histogram '{hist_name}' in ROOT file. "
                f"Tried path: {tried[0]}"
            )


# ----------------------------------------------------------------------
# Build tables and submission
# ----------------------------------------------------------------------

def build_table_for_region(reader, cfg):
    root_dir = cfg["root_dir"]
    data_dict = read_hist_1d(reader, root_dir, cfg["data_hist"])

    x_var = Variable(
        OBSERVABLE_NAME,
        is_independent=True,
        is_binned=True,
        units=OBSERVABLE_UNITS,
    )
    x_var.values = data_dict["x_edges"]

    table = Table(cfg["name"])
    table.location = cfg["location"]
    table.description = cfg["description"]

    # Keywords: common for both regions
    table.keywords["cmenergies"] = [CM_ENERGY_GEV]
    table.keywords["reactions"] = ["P P --> Z GAMMA"]
    table.keywords["observables"] = ["DSIG/DPT"]
    table.keywords["phrases"] = [
        "Differential cross section",
        "Z to invisible",
        "High-pt photon",
        "Run 2",
    ]

    table.add_variable(x_var)
    table.add_variable(make_data_variable("Observed", data_dict))

    for label, hist_name in cfg.get("primary_mc", []):
        mc_dict = read_hist_1d(reader, root_dir, hist_name)
        table.add_variable(make_mc_variable(label, mc_dict))

    for label, hist_name in cfg.get("other_mc", []):
        mc_dict = read_hist_1d(reader, root_dir, hist_name)
        table.add_variable(make_mc_variable(label, mc_dict))

    for label, hist_name in cfg.get("extra_mc", []):
        extra_dict = read_hist_1d(reader, root_dir, hist_name)
        table.add_variable(make_mc_variable(label, extra_dict))

    return table


def build_submission():
    sub = Submission()

    # Short top-level description of the analysis
    sub.comment = (
        f"{PAPER_TITLE}. CMS analysis {CMS_ANALYSIS_ID}. "
        "Z→νν with an associated high-$p_{T}$ photon, using 138 fb$^{-1}$ "
        "of pp collisions at $\\sqrt{s}=13$ TeV. "
        "The tables in this submission provide the post-fit reconstruction-level "
        "$p_{T}^{\\gamma}$ spectra in the EB and EE signal regions corresponding "
        "to Figure 4 of the paper."
    )

    # Optional abstract file (if you provide one)
    if ABSTRACT_FILE is not None and os.path.exists(ABSTRACT_FILE):
        sub.read_abstract(ABSTRACT_FILE)

    reader = RootFileReader(ROOT_FILENAME)
    for cfg in REGIONS:
        table = build_table_for_region(reader, cfg)
        sub.add_table(table)
    return sub


def main():
    sub = build_submission()
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    sub.create_files(OUTPUT_DIR)
    print(f"HEPData submission written to: {OUTPUT_DIR}")


# ----------------------------------------------------------------------
# Simple YAML readback/validation
# ----------------------------------------------------------------------

def validate_hepdata_output(output_dir=OUTPUT_DIR):
    """Basic sanity checks on generated YAML (bin counts vs. values)."""
    try:
        import yaml
    except ImportError:
        print("PyYAML is not available; cannot parse YAML for validation.")
        return

    yaml_files = sorted(glob.glob(os.path.join(output_dir, "*.yaml")))
    if not yaml_files:
        print(f"No YAML files found in {output_dir}")
        return

    print(f"\nValidation of HEPData YAML files in: {output_dir}")
    for path in yaml_files:
        with open(path, "r") as f:


_IncompleteInputError: incomplete input (3565459980.py, line 296)

In [17]:
main()
validate_hepdata_output()

HEPData submission written to: hepdata_output

Validation of HEPData YAML files in: hepdata_output

=== figure4_eb_ptgamma.yaml ===
  independent_variables: 1, dependent_variables: 10
  number of bins (from first independent variable): 11
    Observed: 11 values -> OK
    Fiducial $Z\gamma$: 11 values -> OK
    $W+\gamma$: 11 values -> OK
    Spikes: 11 values -> OK
    $e\to\gamma$: 11 values -> OK
    Jet fakes: 11 values -> OK
    Out-of-acceptance $Z(\nu\nu)\gamma$: 11 values -> OK
    Minor backgrounds: 11 values -> OK
    Total background: 11 values -> OK
    Predicted: 11 values -> OK

=== figure4_ee_ptgamma.yaml ===
  independent_variables: 1, dependent_variables: 10
  number of bins (from first independent variable): 7
    Observed: 7 values -> OK
    Fiducial $Z\gamma$: 7 values -> OK
    $W+\gamma$: 7 values -> OK
    $e\to\gamma$: 7 values -> OK
    Beam halo: 7 values -> OK
    Jet fakes: 7 values -> OK
    Out-of-acceptance $Z(\nu\nu)\gamma$: 7 values -> OK
    Minor back